In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [13]:
home_data_path = '/Users/dadaya/Desktop/home-data-for-ml-course/train.csv'
home_data = pd.read_csv(home_data_path)
home_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

Часть 1. Простейшая модель

In [6]:
home_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [14]:
y = home_data.SalePrice # целевой атрибут, который будем предсказывать

feature_names = ['LotArea', 'YearBuilt', '1stFlrSF',  '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']

X = home_data[feature_names] # атрибуты, по к   §ASоторым мы будем делать предсказание целевого атрибута

In [17]:
X.describe()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,1971.267808,1162.626712,346.992466,1.565068,2.866438,6.517808
std,9981.264932,30.202904,386.587738,436.528436,0.550916,0.815778,1.625393
min,1300.000000,1872.000000,334.000000,0.000000,0.000000,0.000000,2.000000
25%,7553.500000,1954.000000,882.000000,0.000000,1.000000,2.000000,5.000000
50%,9478.500000,1973.000000,1087.000000,0.000000,2.000000,3.000000,6.000000
75%,11601.500000,2000.000000,1391.250000,728.000000,2.000000,3.000000,7.000000
max,215245.000000,2010.000000,4692.000000,2065.000000,3.000000,8.000000,14.000000


In [18]:
X.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
1,8450,2003,856,854,2,3,8
2,9600,1976,1262,0,2,3,6
3,11250,2001,920,866,2,3,6
4,9550,1915,961,756,1,3,7
5,14260,2000,1145,1053,2,4,9


Шаги на пути к созданию и использованию модели таковы:

$\cdot$**Выбор алгоритма**: Модель какого типа это будет?(DecisionTreeRegressor, RandomForest,...). Выбор модели зависит от решаемой задачи, используемых признаков и требований по сложности (например, если модель будет дальше внедряться в Excel, то Дерево решений или AdaBoost не подойдут). При выборе модели обязательно принять во внимание следующие факторы:

$\cdot$ Достаточность данных (обычно, сложные модели требуют большого количества данных).

$\cdot$ Обработка пропусков (некоторые алгоритмы не умеют обрабатывать пропуски).

$\cdot$ Формат данных (для части алгоритмов потребуется конвертация данных).

$\cdot$**Нормализовать**: Объеденить зависимости из предоставленных данных. Это ключ моделирования.

$\cdot$**Предсказать**: Предсказать

$\cdot$**Оценить**: Определить насколько точны результаты моделирования

In [21]:
simpliest_model_decision_tree = DecisionTreeRegressor(random_state = 1)
simpliest_model_decision_tree.fit(X, y)
predictions = simpliest_model_decision_tree.predict(X)
print(predictions)

[208500. 181500. 223500. ... 266500. 142125. 147500.]


In [22]:
print(y.head())
print(simpliest_model_decision_tree.predict(X.head()))

Id
1    208500
2    181500
3    223500
4    140000
5    250000
Name: SalePrice, dtype: int64
[208500. 181500. 223500. 140000. 250000.]


Часть 2. Валидация модели

In [15]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1) # Создали тренировочные данные и валидационные данные, на тренировочных данных модель будет обучаться, а на валидвационныъ

In [16]:
simpliest_model_decision_tree = DecisionTreeRegressor(random_state = 1)
simpliest_model_decision_tree.fit(train_X, train_y)
val_predictions = simpliest_model_decision_tree.predict(val_X)

In [17]:
print(val_predictions)
print(val_y.head())

[186500. 184000. 130000.  92000. 164500. 220000. 335000. 144152. 215000.
 262000. 180000. 121000. 175900. 210000. 248900. 131000. 100000. 149350.
 235000. 156000. 149900. 265979. 193500. 377500. 100000. 162900. 145000.
 180000. 582933. 146000. 140000.  91500. 112500. 113000. 145000. 312500.
 110000. 132000. 305000. 128000. 162900. 115000. 110000. 124000. 215200.
 180000.  79000. 192000. 282922. 235000. 132000. 325000.  80000. 237000.
 208300. 100000. 120500. 162000. 153000. 187000. 185750. 335000. 129000.
 124900. 185750. 133700. 127000. 230000. 146800. 157900. 136000. 153575.
 335000. 177500. 143000. 202500. 168500. 105000. 305900. 192000. 190000.
 140200. 134900. 128950. 213000. 108959. 149500. 190000. 175900. 160000.
 250580. 157000. 120500. 147500. 118000. 117000. 110000. 130000. 148500.
 148000. 190000. 130500. 127000. 120500. 135000. 168000. 176432. 128000.
 147000. 260000. 132000. 129500. 171000. 181134. 227875. 189000. 282922.
  94750. 185000. 194000. 159000. 279500. 290000. 13

In [20]:
print(mean_absolute_error(val_y, val_predictions))

29652.931506849316


**Часть 3** Переподгон и недоподгон

In [22]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
result_mae = {}
for i, max_leaf_nodes in enumerate(candidate_max_leaf_nodes):
    result_mae[i] = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)

best_tree_size = candidate_max_leaf_nodes[get_key(result_mae, min(result_mae.values()))]
best_tree_size

100

In [23]:
final_model = DecisionTreeRegressor(max_leaf_nodes = 100)

final_model.fit(X, y)

DecisionTreeRegressor(max_leaf_nodes=100)

**Часть 4** Случайные леса

In [25]:
rf_model = RandomForestRegressor(random_state = 1)

rf_model.fit(train_X, train_y)

rf_val_mae = mean_absolute_error(val_y, rf_model.predict(val_X))

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))

Validation MAE for Random Forest Model: 21857.15912981083


**Часть 5** Отсутствующие значения

Существует три подхода к пропущенным значениям:
1. Простое удаление любых столбцов, содержащих значения NaN
2. Вставка в ячейки со значениями NaN среднего по всему столбцу значения
3. Вставка среднего с созданием дополнительного столбца, который показывает, куда были вставлены средние значения

In [26]:
X_full = pd.read_csv(home_data_path, index_col='Id')
X_test_full = pd.read_csv(home_data_path, index_col='Id')

X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [27]:
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


Удаление столбцов

In [28]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

columns_with_null = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

reduced_X_train = X_train.drop(columns = columns_with_null)
reduced_X_valid = X_valid.drop(columns = columns_with_null)

In [29]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


Замена на среднее

In [30]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [31]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


In [32]:
final_X_train = pd.DataFrame(imputer.fit_transform(X_train.drop(columns = 'GarageYrBlt')))
final_X_valid = pd.DataFrame(imputer.transform(X_valid.drop(columns = 'GarageYrBlt')))
final_X_train.columns = X_train.drop(columns = 'GarageYrBlt').columns
final_X_valid.columns = X_valid.drop(columns = 'GarageYrBlt').columns

model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
18014.59256849315


In [36]:
final_X_test = pd.DataFrame(imputer.fit_transform(X_test.drop(columns = 'GarageYrBlt')))
final_X_test.columns = X_test.drop(columns = 'GarageYrBlt').columns

preds_test = model.predict(final_X_valid)
print(preds_test)
print(y_valid)

[213288.55       151523.68       102252.73       224203.3
  90304.72       111860.75       264741.77       122568.58
 508368.1        156339.71       194721.7        144151.
 221301.         113085.69       128268.42       148109.25
 220883.14       119104.08       143429.12       181093.13
 125936.82       144732.41666667 124710.84       163819.96
 178949.8        221529.11       171571.82        85142.71
 335265.31       120072.52       137405.32       192742.65
 139309.55       288713.46       328443.92       181870.55
 284568.25       126864.         231743.28       304830.53
 220925.44       120814.82       180972.19       292175.16
 318432.78       144946.04       128993.         128095.25
 169657.5        102706.36       367494.93       144336.
 167368.32        84663.08       238578.25       112456.92
 143682.15       249148.92       143114.          98994.55
 143151.37       133369.75       145040.5        150217.
 205525.78       155883.5        125311.82       216707.25
 118

**Часть 6** Категориальные признаки

Существует три основных подхода к работе с категориальными признаками:
1. Удалить столбцы с категориальными признаками
2. Изменить каждый экземпляр категориального признака на соответствующиее ему по смыслу число, например - "Никогда, редко, обычно, часто, всегда" - "0, 1, 2, 3, 4"
3. В случае если нельзя применить второй метод - а именно если категориальный признак нельзя разложить по разным смысловым значениям, можно создать столбец, соответствующий каждому новому экземпляру признака и отметить в нем 0 и 1 вхождения в каждой строке

In [39]:
X = pd.read_csv(home_data_path, index_col='Id') 
X_test = pd.read_csv(home_data_path, index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll drop columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

Удаление столбцов

In [40]:
drop_X_train = X_train.select_dtypes(exclude = ['object'])
drop_X_valid = X_valid.select_dtypes(exclude = ['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
17837.82570776256


Числовая замена - Ordinal Encoding

In [43]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

good_label_cols = [col for col in object_cols if set(X_valid[col]).issubset(set(X_train[col]))]

bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be ordinal encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['RoofMatl', 'Functional', 'Condition2']


In [44]:
from sklearn.preprocessing import OrdinalEncoder

label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

ordinal_encoder = OrdinalEncoder()
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])

In [45]:
print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
17098.01649543379


One-Hot Encoding

In [46]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

In [47]:
from sklearn.preprocessing import OneHotEncoder

# Use as many lines of code as you need!
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [48]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
17525.345719178084


**Часть 7** Пайплайны

**Вступление**

Пайплайны - это более простой способ изображать код модели и пре-процессинга организованным. В частности, пайплайн связывает шаги моделирования и пре-процессинга для того чтобы использовать эту связку как одно целое, в один шаг.

У использования пайплайнов есть несколько плюсов. Среди них:

**Более чистый код**: Отсылки к данным на каждом шагу препроцессинга могут быть очень грязыми. Используя пайплайн, не нужно будет постоянно следить за тренировочнойи валидационной датой на каждом шагу.
**Меньше багов**: There are fewer opportunities to misapply a step or forget a preprocessing step.
**Easier to Productionize**: It can be surprisingly hard to transition a model from a prototype to something deployable at scale. We won't go into the many related concerns here, but pipelines can help.
**More Options for Model Validation**: You will see an example in the next tutorial, which covers cross-validation.

**Example**

As in the previous tutorial, we will work with the Melbourne Housing dataset.

We won't focus on the data loading step. Instead, you can imagine you are at a point where you already have the training and validation data in X_train, X_valid, y_train, and y_valid.

In [50]:
from sklearn.pipeline import Pipeline

In [51]:
train_data = pd.read_csv(home_data_path, index_col='Id')
test_data = pd.read_csv(home_data_path, index_col='Id')

# Remove rows with missing target, separate target from predictors
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = train_data.SalePrice              
train_data.drop(['SalePrice'], axis=1, inplace=True)

# Select numeric columns only
numeric_cols = [cname for cname in train_data.columns if train_data[cname].dtype in ['int64', 'float64']]
X = train_data[numeric_cols].copy()
X_test = test_data[numeric_cols].copy()

In [52]:
my_pipeline = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', RandomForestRegressor(n_estimators=50, random_state=0))
])

In [53]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("Average MAE score:", scores.mean())

Average MAE score: 18276.410356164386


In [54]:
def get_score(n_estimators):
    my_pipeline = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', RandomForestRegressor(n_estimators=n_estimators, random_state=0))])
    
    score = -1 * cross_val_score(my_pipeline, X, y, cv=3, scoring='neg_mean_absolute_error')
    return score.mean()

In [55]:
n_estimators = [50 * i for i in range(1, 9)]
results = [get_score(n_estimators[i]) for i in range(8)]
print(results)

[18353.8393511688, 18395.2151680032, 18288.730020956387, 18248.345889801505, 18255.26922247291, 18275.241922621914, 18270.29183308043, 18270.197974402367]
